## Introduction to BGP Analysis using Batfish

Network engineers routinely need to validate BGP configuration and session status in the network. This is commonly done by connecting to multiple network devices and executing a series of `show ip bgp` commands. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes BGP analysis extremely simple by providing an easy-to-query, centralized view of routing tables in the network. 

In this notebook, we will show how you can extract BGP configuration and session status information with Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-bgp-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

/Users/corinaminer/Batfish/pybatfish/pybatfish/client/commands.py:50: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "bgp_network"
SNAPSHOT_NAME = "bgp_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'bgp_snapshot'

now tell them about the network and the issue: Interface utilization on core routers is badly distributed

Traffic from dept disproportionately hits core2 instead of core1 *(reason: dist2 doesn't have a route to core1)*

## BGP process configuration
First, let's ensure that BGP is configured on the routers in question. The `BgpProcessConfiguration` question lists BGP processes configured on a given set of nodes, or on all nodes if no parameters are given.

In [3]:
bfq.bgpProcessConfiguration(nodes='as2core1|as2dist2').answer().frame()

,Node,VRF,Router_ID,Cluster_IDs,Multipath_EBGP,Multipath_IBGP,Tie_Breaker,Neighbors,Route_Reflector,Multipath_Match_Mode
0,as2dist2,default,2.1.3.2,None,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '2.34.201.4/32']",False,EXACT_PATH
1,as2core1,default,2.1.2.1,['2.1.2.1'],True,True,ARRIVAL_ORDER,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",True,EXACT_PATH


The results of `BgpProcessConfiguration` show that `as2core1` and `as2dist2` do have BGP configured. We can also see that `as2core1` has a `Cluster ID`, which means it is configured to be a route reflector for some of its BGP peers.

## BGP peer configuration
Next we can look for any issues with the individual BGP peers configured on `as2core1` and `as2dist2`. The `BgpPeerConfiguration` question shows all BGP peers configured on a given set of nodes, or on all nodes if no parameters are given.

In [4]:
# BgpPeerConfiguration shows all configured BGP peers
bfq.bgpPeerConfiguration(nodes='as2core1|as2dist2').answer().frame()

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_IP,Route_Reflector_Client,Peer_Group,Import_Policy,Export_Policy,Send_Community
0,as2core1,default,2,2.1.2.1,2,2.1.1.1,True,as2,[],[],True
1,as2dist2,default,2,2.1.3.2,2,2.1.2.1,False,as2,[],[],True
2,as2dist2,default,2,2.1.3.2,2,2.1.2.2,False,as2,[],[],True
3,as2core1,default,2,2.1.2.1,2,2.1.3.1,True,as2,[],[],True
4,as2dist2,default,2,2.34.201.3,65001,2.34.201.4,False,dept,['dept_to_as2dist'],['as2dist_to_dept'],True
5,as2core1,default,2,2.1.2.1,2,2.1.1.2,True,as2,[],[],True
6,as2core1,default,2,2.1.2.1,2,2.1.3.2,True,as2,[],[],True


Rows 1 and 6 appear to be correctly configured to peer with each other.

## BGP session status
Next, let's see if the BGP session between `as2core1` and `as2dist2` will get established. The `BgpSessionStatus` question shows which compatible BGP peers establish sessions. Unlike the previous questions, this question relies on Batfish's generated data plane to provide information that would normally only be accessible by querying the live network, so it will take longer to run the first time while the data plane gets built.

In [ ]:
bfq.bgpSessionStatus(nodes='as2core1|as2dist2').answer().frame()

Now we can see that the BGP session did *not* get established between the BGP peers on `as2core1` and `as2dist2`, even though they seemed compatible based on the results of `BgpPeerConfiguration`.

## BGP session compatibility
Did we miss something when we looked at the results of `BgpPeerConfiguration`? The `BgpSessionCompatibility` question gives a direct answer to whether BGP peers are compatible, and better insight into why incompatible peers are not compatible.

In [ ]:
bfq.bgpSessionCompatibility(nodes='as2core1|as2dist2').answer().frame()

Now we can be confident that the BGP peers between `as2core1` and `as2dist2` are correctly configured. There must be some other problem preventing the session from being established.

While we're looking at `BgpSessionCompatibility`, let's check the rest of the network for any BGP peers that are not compatibly configured. We will limit the output to peers whose configured status is *not* `UNIQUE_MATCH`.

In [ ]:
all_session_compatibility = bfq.bgpSessionCompatibility().answer().frame()
all_session_compatibility[all_session_compatibility['Configured_Status'] != 'UNIQUE_MATCH']

Of these results, the first two have no local IP configured, so will be unable to establish sessions: almost certainly an oversight that should be fixed. The third has a remote IP outside of the network Batfish is modeling, so it may be able to establish a session in the live network, but Batfish doesn't have enough information to be sure. Here is the full list of possible configured statuses:
- `LOCAL_IP_UNKNOWN_STATICALLY` when no local IP is configured for an IBGP or EBGP multihop session
- `NO_LOCAL_IP` when no local IP is configured for other types of sessions
- `INVALID_LOCAL_IP` when the local IP is not associated with a known interface
- `NO_REMOTE_AS` when no remote AS is configured
- `UNKNOWN_REMOTE` when no remote IP is configured, or the remote IP is not associated with a known interface
- `HALF_OPEN` when no remote peer is configured compatibly
- `MULTIPLE_REMOTES` when multiple remote peers are configured compatibly
- `UNIQUE_MATCH` when the peer is compatibly configured with one remote peer
- `DYNAMIC_LISTEN` for a correctly configured passive peer

These three results do not offer an explanation for the problem at hand, so let's keep looking.

## Check routes
Now we know the BGP peers on `as2core1` and `as2dist2` are compatible, but the session is not established. Let's make sure the routes that should be used to establish the session are present.

In [ ]:
# Show loopback routes for as2core1
routes = bfq.routes(nodes="as2core1|as2dist2").answer().frame()
routes[routes['Network'].apply(lambda x: '/32' in x)]

Find that loopback is not advertised (missing from routes), thus solving the mystery

***
### Get involved with the Batfish community

Join our community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [Github](https://github.com/batfish/batfish). 